In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#https://www.kaggle.com/datasets/vivek468/superstore-dataset-final?select=Sample+-+Superstore.csv
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/csv/Data Mining/Sample - Superstore.csv',encoding='cp1252')

In [ ]:
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [ ]:
rec=data[['Customer Name','Product Name','Quantity']]
g=rec.groupby(['Customer Name','Product Name']).sum()
rec_data=g.add_suffix('').reset_index()

In [ ]:
d=rec_data.groupby(['Product Name']).max().reset_index()
max={value:key for key in d['Quantity'] for value in d['Product Name']}
d1=rec_data.groupby(['Product Name']).min().reset_index()
min={value:key for key in d1['Quantity'] for value in d1['Product Name']}

In [ ]:
L1=rec_data['Product Name'].tolist()
L=rec_data['Quantity'].tolist()
for i in range(len(L)):
  L[i]=((L[i]-min[L1[i]])/(max[L1[i]]-min[L1[i]]))
rec_data['Quantity']=L

In [ ]:
from sklearn.utils import shuffle
rec_data=shuffle(rec_data)

In [ ]:
rec_data.head(2)

,Customer Name,Product Name,Quantity
6200,Mark Packer,Xerox 205,0.857143
2174,Craig Yedwab,"#10 Gummed Flap White Envelopes, 100/Box",0.142857


In [ ]:
ratings_utility_matrix = rec_data.pivot_table(values='Quantity', index='Customer Name', columns='Product Name', fill_value=0)

In [ ]:
X = ratings_utility_matrix.T

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1850 entries, "While you Were Out" Message Book, One Form per Page to netTALK DUO VoIP Telephone Service
Columns: 793 entries, Aaron Bergman to Zuschuss Donatelli
dtypes: float64(793)
memory usage: 11.2+ MB


In [ ]:
#https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/
import sklearn
from sklearn.decomposition import TruncatedSVD
import numpy as np
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)

(1850, 10)

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [ ]:
def Recommend(p):
  product_names = list(X.index)
  product_ID = product_names.index(p)
  correlation_product_ID = correlation_matrix[product_ID]
  Recommend = list(X.index[correlation_product_ID > 0.90])
  if p in Recommend:
    Recommend.remove(p) 
  return Recommend[0:3]
  

In [ ]:
Recommendations={}
for prod in X.index :
  Recommendations[prod]=' / '.join(Recommend(prod))


In [ ]:
cos_rec={}
for cos in data['Customer Name'].unique():
  l=['','']
  for prod in data[data['Customer Name']==cos]['Product Name'].unique():
    l[0]+=' / '+prod
    l[1]+=' // '+ Recommendations[prod]
  cos_rec[cos]=l


In [ ]:
dictlist=[]
for key, value in cos_rec.items():
    temp = [key,value[0],value[1]]
    dictlist.append(temp)

In [ ]:
customers_Recommendations=pd.DataFrame(dictlist, columns=['Customer','Bought Products','Recommendations'])

In [ ]:
customers_Recommendations.to_csv('/content/drive/MyDrive/Colab_Notebooks/csv/Data Mining/Recommendations.csv')

In [ ]:
def Customer_Recommender(customer_name):
  customers_Recommendations[customers_Recommendations['Customer']==customer_name]

In [ ]:
def recommend_for(cos):
    l1=customers_Recommendations[customers_Recommendations['Customer']==cos]['Recommendations'].tolist()[0].split(' // ')[1:]
    l2=customers_Recommendations[customers_Recommendations['Customer']==cos]['Bought Products'].tolist()[0].split(' // ')
    l11=[i.strip() for i in l1]
    while '' in l11:
        l11.remove('')
    l22=[i.strip() for i in l2 ]
    while '' in l22:
        l22.remove('')
    print(f" based on these purchases : \n {l22} \n the recommendations for {cos}  are :\n {l11} ")